# Q-SITE 2024: IBM Quantum Hackathon

## Lab 2: Quantum Enigma - Four-square chessboard



This challenge is based on the Quantum Enigmas produced by Institut Quantique at the Université de Sherbrooke. Watch this video to see the details of the Quantum Enigma entitled ***The Four-Square Chessboard***: https://youtu.be/UuVbtFXOEKQ?feature=shared .

In this challenge, you will code the solution to the enigma using Qiskit.

The code required to complete the exercise should be typed under the line which has the `## WRITE YOUR CODE BELOW HERE ##` comment. Avoid making changes to any provided code as this could alter how other code in the notebook functions. You will submit answers to the grader methods that are called in this notebook. This lab has 4 graded exercises.

The Four-Square Chessboard enigma is summarized as follows. Alice and Bob play a game in which Bob must find the location of a key hidden under one of the squares of a chessboard. Each chessboard square also has a coin with one side labeled 0 and the other side labeled 1, and each coin is placed randomly on one of these two sides. Alice must indicate the location of the hidden key to Bob by only flipping one coin on the board. Bob is provided with no information except the state of the chessboard after Alice flips the coin (i.e. he does not know which coin Alice flipped).

Below is an example of the chessboard with the coin values shown in the video, with the example location of the key in red. However, the board could be in any configuration. In this case, there are 64 possible configurations, i.e. 16 possible coin configurations $\times$ 4 possible key locations.

<center>
<table border=1>
    <colgroup>
        <col span="1" style="width:50px; text-align: center;">
        <col span="1" style="width:50px; text-align: center;">
    </colgroup>
    <tbody>
        <tr style="height:50px"><td>0</td><td bgcolor='red'>1</span></td></tr>
        <tr style="height:50px"><td>1</td><td>0</td></tr>
    </tbody>
</table>
</center>

Alice and Bob come into the game with a strategy to allow Bob to find the key given the rules of the game. It is based on computing parities of the coin values on the board. Alice will flip a coin based on her calculation of the partities of the key location expressed in binary, and a focus square location that is chosen by computing the parities of coins on the board. Bob will repeat this parity calculation of the coins on the board in order to find the location of the key.

To validate this strategy, we can use a quantum computer to represent all of the possible configurations simultaneously and compute this strategy. 

In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.primitives import StatevectorSampler

from grader.qsite24_ibm_grader import *

### Part 1: Construct circuit for four-square chessboard

Use this initial code that sets up the quantum circuit with a set of quantum and classical registers. This setup is a bit different from video where the qubits were not divided into named register and were instead specified solely by their index number. This may become cumbersome when scaling up the circuit as you will do in Part 2. Register allow your code to be more readable.

The registers below represent the following:

- `q_reg_squares`: represents the squares of the chessboard. In the binary mapping of the square numbers, the first bit represents the row of the square while the second bit represents the column of the square.
- `q_reg_key`: the key location in binary.
- `q_reg_afocus`: Alice's computed focus square in binary.
- `q_reg_bfocus`: Bob's computed focus square in binary.
- `c_reg_key_bfocus`: The measured key location (first pair of qubits) and Bob's focus square (second pair of qubits). These are measured into the same classical register for convenience to clearly see the result in the end.

In [125]:
q_reg_squares = QuantumRegister(4, name='squares')
q_reg_key = QuantumRegister(2, name='key')
q_reg_afocus = QuantumRegister(2, name='afocus')
q_reg_bfocus = QuantumRegister(2, name='bfocus')

c_reg_key_bfocus = ClassicalRegister(4, name='c_key_bfocus')

qc = QuantumCircuit(q_reg_squares, q_reg_key, q_reg_afocus, q_reg_bfocus, c_reg_key_bfocus)
qc.draw()

squares_0: 
                
     squares_1: 
                
     squares_2: 
                
     squares_3: 
                
         key_0: 
                
         key_1: 
                
      afocus_0: 
                
      afocus_1: 
                
      bfocus_0: 
                
      bfocus_1: 
                
c_key_bfocus: 4/

#### Step 1

In the first step, place the qubits of the square and key registers into equal superpositions of 0 and 1. This will effectively represent all possible combinations of coin values and key locations.

In [126]:
## WRITE YOUR CODE BELOW HERE ##

# apply H gate into squares register qubits 
for qubit in q_reg_squares:
    qc.h(qubit)

# apply H gate into key qubits
for qubit in q_reg_key:
    qc.h(qubit)

qc.draw()

┌───┐
     squares_0: ┤ H ├
                ├───┤
     squares_1: ┤ H ├
                ├───┤
     squares_2: ┤ H ├
                ├───┤
     squares_3: ┤ H ├
                ├───┤
         key_0: ┤ H ├
                ├───┤
         key_1: ┤ H ├
                └───┘
      afocus_0: ─────
                     
      afocus_1: ─────
                     
      bfocus_0: ─────
                     
      bfocus_1: ─────
                     
c_key_bfocus: 4/═════

#### Step 2

In the next step, the focus square is selected by computing the partities of the bottom row and right-most column.

In [127]:
## WRITE YOUR CODE BELOW HERE ##

# Compute row parity (bottom row: qubits 1 and 3 of q_reg_squares)
qc.cx(q_reg_squares[1], q_reg_afocus[0])  
qc.cx(q_reg_squares[3], q_reg_afocus[0]) 

# Compute column parity (right-most column: qubits 2 and 3 of q_reg_squares)
qc.cx(q_reg_squares[2], q_reg_afocus[1])  
qc.cx(q_reg_squares[3], q_reg_afocus[1]) 

qc.draw()


┌───┐                    
     squares_0: ┤ H ├────────────────────
                ├───┤                    
     squares_1: ┤ H ├──■─────────────────
                ├───┤  │                 
     squares_2: ┤ H ├──┼────■────────────
                ├───┤  │    │            
     squares_3: ┤ H ├──┼────┼────■────■──
                ├───┤  │    │    │    │  
         key_0: ┤ H ├──┼────┼────┼────┼──
                ├───┤  │    │    │    │  
         key_1: ┤ H ├──┼────┼────┼────┼──
                └───┘┌─┴─┐  │  ┌─┴─┐  │  
      afocus_0: ─────┤ X ├──┼──┤ X ├──┼──
                     └───┘┌─┴─┐└───┘┌─┴─┐
      afocus_1: ──────────┤ X ├─────┤ X ├
                          └───┘     └───┘
      bfocus_0: ─────────────────────────
                                         
      bfocus_1: ─────────────────────────
                                         
c_key_bfocus: 4/═════════════════════════

#### Step 3

Now Alice must determine which coin to flip. This is done by computing the location of the square by computing the parities of each bit between the key location and the focus square registers. When done, the square with the coin to flip will be stored in Alice's focus register.

In [128]:
## WRITE YOUR CODE BELOW HERE ##

qc.cx(q_reg_key[0], q_reg_afocus[0])  
qc.cx(q_reg_key[1], q_reg_afocus[1])  

qc.draw()

┌───┐                              
     squares_0: ┤ H ├──────────────────────────────
                ├───┤                              
     squares_1: ┤ H ├──■───────────────────────────
                ├───┤  │                           
     squares_2: ┤ H ├──┼────■──────────────────────
                ├───┤  │    │                      
     squares_3: ┤ H ├──┼────┼────■────■────────────
                ├───┤  │    │    │    │            
         key_0: ┤ H ├──┼────┼────┼────┼────■───────
                ├───┤  │    │    │    │    │       
         key_1: ┤ H ├──┼────┼────┼────┼────┼────■──
                └───┘┌─┴─┐  │  ┌─┴─┐  │  ┌─┴─┐  │  
      afocus_0: ─────┤ X ├──┼──┤ X ├──┼──┤ X ├──┼──
                     └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐
      afocus_1: ──────────┤ X ├─────┤ X ├─────┤ X ├
                          └───┘     └───┘     └───┘
      bfocus_0: ───────────────────────────────────
                                                   
      bfocus_1: ───────────────────────────────────
                                                   
c_key_bfocus: 4/═══════════════════════════════════

In [129]:
## SUBMIT CIRCUIT TO GRADER
qsite24_grader_lab2ex1a(qc)

Congratulations! 🎉 Your answer is correct.


#### Step 4

Now flip the coins according to the value in Alice's focus register.

In [130]:
## WRITE YOUR CODE BELOW HERE ##

# toffoli gate with a_focus as control and square 3 as target
qc.ccx(q_reg_afocus[1], q_reg_afocus[0], q_reg_squares[3])

# not gate on focus 0
qc.x(q_reg_afocus[0])
# toffoli gate with a_focus as control and square 2 as target
qc.ccx(q_reg_afocus[1], q_reg_afocus[0], q_reg_squares[2])
# not gate on focus 0
qc.x(q_reg_afocus[0])

# not gate on focus 1
qc.x(q_reg_afocus[1])
# toffoli gate with a_focus as control and square 1 as target
qc.ccx(q_reg_afocus[1], q_reg_afocus[0], q_reg_squares[1])
# not gate on focus 1
qc.x(q_reg_afocus[1])

# not gate on focus 1
qc.x(q_reg_afocus[1])
# not gate on focus 0
qc.x(q_reg_afocus[0])
# toffoli gate with a_focus as control and square 0 as target
qc.ccx(q_reg_afocus[1], q_reg_afocus[0], q_reg_squares[0])
# not gate on focus 0
qc.x(q_reg_afocus[0])
# not gate on focus 1
qc.x(q_reg_afocus[1])


In [131]:
## SUBMIT CIRCUIT TO GRADER
qsite24_grader_lab2ex1b(qc)

Congratulations! 🎉 Your answer is correct.


#### Step 5

Now it's Bob's turn. To determine the coin location, compute the parity of the last row and column just as Alice did previously. Bob's computation of the key location should be stored in his focus register. Once this is done, measure out the key register and Bob's focus register into the same classical register (this code is given).

In [120]:
## WRITE YOUR CODE BELOW HERE ##

qc.cx(q_reg_squares[1], q_reg_bfocus[0])
qc.cx(q_reg_squares[3], q_reg_bfocus[0])
qc.cx(q_reg_squares[2], q_reg_bfocus[1])
qc.cx(q_reg_squares[3], q_reg_bfocus[1])

qc.measure(q_reg_key, c_reg_key_bfocus[0:2])
qc.measure(q_reg_bfocus, c_reg_key_bfocus[2:4])

qc.draw()


┌───┐                                                        »
     squares_0: ┤ H ├────────────────────────────────────────────────────────»
                ├───┤                                                        »
     squares_1: ┤ H ├──■─────────────────────────────────────────────────────»
                ├───┤  │                                           ┌───┐     »
     squares_2: ┤ H ├──┼────■──────────────────────────────────────┤ X ├─────»
                ├───┤  │    │                         ┌───┐        └─┬─┘     »
     squares_3: ┤ H ├──┼────┼────■────■───────────────┤ X ├──────────┼───────»
                ├───┤  │    │    │    │            ┌─┐└─┬─┘          │       »
         key_0: ┤ H ├──┼────┼────┼────┼────■───────┤M├──┼────────────┼───────»
                ├───┤  │    │    │    │    │       └╥┘  │  ┌─┐       │       »
         key_1: ┤ H ├──┼────┼────┼────┼────┼────■───╫───┼──┤M├───────┼───────»
                └───┘┌─┴─┐  │  ┌─┴─┐  │  ┌─┴─┐  │   ║   │  └╥┘┌───┐  │  ┌───┐»
      afocus_0: ─────┤ X ├──┼──┤ X ├──┼──┤ X ├──┼───╫───■───╫─┤ X ├──■──┤ X ├»
                     └───┘┌─┴─┐└───┘┌─┴─┐└───┘┌─┴─┐ ║   │   ║ └───┘  │  ├───┤»
      afocus_1: ──────────┤ X ├─────┤ X ├─────┤ X ├─╫───■───╫────────■──┤ X ├»
                          └───┘     └───┘     └───┘ ║       ║           └───┘»
      bfocus_0: ────────────────────────────────────╫───────╫────────────────»
                                                    ║       ║                »
      bfocus_1: ────────────────────────────────────╫───────╫────────────────»
                                                    ║       ║                »
c_key_bfocus: 4/════════════════════════════════════╩═══════╩════════════════»
                                                    0       1                »
«                                              ┌───┐          
«     squares_0: ──────────────────────────────┤ X ├──────────
«                     ┌───┐                    └─┬─┘          
«     squares_1: ─────┤ X ├───────■──────────────┼────────────
«                     └─┬─┘       │              │            
«     squares_2: ──■────┼─────────┼──────────────┼────────────
«                  │    │         │              │            
«     squares_3: ──┼────┼─────────┼─────────■────┼────■───────
«                  │    │         │         │    │    │       
«         key_0: ──┼────┼─────────┼─────────┼────┼────┼───────
«                  │    │         │         │    │    │       
«         key_1: ──┼────┼─────────┼─────────┼────┼────┼───────
«                  │    │  ┌───┐  │         │    │    │  ┌───┐
«      afocus_0: ──┼────■──┤ X ├──┼─────────┼────■────┼──┤ X ├
«                  │    │  ├───┤  │  ┌───┐  │    │    │  ├───┤
«      afocus_1: ──┼────■──┤ X ├──┼──┤ X ├──┼────■────┼──┤ X ├
«                  │       └───┘┌─┴─┐└───┘┌─┴─┐ ┌─┐   │  └───┘
«      bfocus_0: ──┼────────────┤ X ├─────┤ X ├─┤M├───┼───────
«                ┌─┴─┐          └───┘     └───┘ └╥┘ ┌─┴─┐ ┌─┐ 
«      bfocus_1: ┤ X ├───────────────────────────╫──┤ X ├─┤M├─
«                └───┘                           ║  └───┘ └╥┘ 
«c_key_bfocus: 4/════════════════════════════════╩═════════╩══
«                                                2         3

#### Step 6

Lastly, run this circuit using `StatevectorSampler` and print out the result.

In [121]:
## WRITE YOUR CODE BELOW HERE ##
sampler = StatevectorSampler()
pub = (qc)
job = sampler.run([pub], shots=1024)



result = job.result()     # Replace None with the result object from the sampler job

You should see in the ```c_key_bfocus``` classical register, the values of the first pair of qubits (key location) and the second pair of qubits (Bob's focus square) always match in each bitstring from the results.

In [122]:
# SUBMIT RESULT TO GRADER HERE (ex1c)
qsite24_grader_lab2ex1c(result)

Congratulations! 🎉 Your answer is correct.


### Part 2: Extension to a 16 square chessboard (4x4)

Now that you have solved the 4 square (2x2) problem, can you scale to a larger chessboard? For this next problem, find a solution to the problem when using a 4x4 chessboard. The rules are the same: Alice can only flip one coin on the board.

<center>
<table border=1>
    <colgroup>
        <col span="1" style="width:50px">
        <col span="1" style="width:50px">
        <col span="1" style="width:50px">
        <col span="1" style="width:50px">
    </colgroup>
    <tbody>
        <tr style="height:50px"><td></td> <td bgcolor='grey'> </td><td> </td><td bgcolor='grey'> </td></tr>
        <tr style="height:50px"><td bgcolor='grey'> </td><td></td><td bgcolor='grey'></td><td> </td></tr>
        <tr style="height:50px"><td> </td><td bgcolor='grey'> </td><td> </td><td bgcolor='grey'> </td></tr>
        <tr style="height:50px"><td bgcolor='grey'> </td><td> </td><td bgcolor='grey'> </td><td> </td></tr>
    </tbody>
</table>
</center>

Note that you'll need to use several more qubits. We suggest that you find a way to minimize the number of qubits in order to keep the memory requirements of the circuit simulation low enough to run on your computer. An initial solution we ran with 28 qubits used nearly 16 GB of RAM and took over 15 minutes to run. HINT: in your 2x2 solution, were there some qubits that you no longer needed at some point in your circuit and could reuse? If so, can you find a way in Qiskit to set them back to zero?

**IMPORTANT NOTE**: construct your circuit to place the final measurements in a classical register with the name ```c_key_focus``` that includes both the key location and Bob's focus square (similar to what was done in the 2x2 solution). Measure the key location in the lower half of the bits in this classical register, and measure Bob's focus square in the upper half of the bits. This is for format expected by the grader.

Simulate your circuit using the ```StatevectorSampler``` and submit the result object from the job object to the grader.

In [12]:
## WRITE YOUR CODE BELOW HERE ##
# Define quantum registers
q_reg_squares = QuantumRegister(16, name='squares')    # 16 qubits
q_reg_key = QuantumRegister(4, name='key')             # 4 qubits
q_reg_focus = QuantumRegister(4, name='focus')         # Reused ancillary qubits
c_reg_key_focus = ClassicalRegister(8, name='c_key_focus')  # For measurement

# Initialize the quantum circuit
qc = QuantumCircuit(q_reg_squares, q_reg_key, q_reg_focus, c_reg_key_focus)

#
qc.draw()





In [11]:
# SUBMIT MEASUREMENT COUNTS TO GRADER HERE (ex2)
qsite24_grader_lab2ex2(result)

Incorrect! The measured results are not as expected


### Congratulations!

You have completed this lab. We hope you enjoyed this experience and learned more about Qiskit!